# Multi-Agent Orchestration with Strands

This notebook demonstrates how to build a system of coordinated AI agents using Strands. 

First, we are going to create and test each individual agent, then we are going to explore a simple use case on how to implement Agent Graphs with Strands.

Agents:

* **get_stock_prices_agent**: Specialized agent to get the latest stock prices. Fetches current and historical stock price data for a given ticker using yahoo finance lib.
* **fin_web_searcher_agent**: Financial researcher agent, focus on search and curated financial information. It uses Tivily API to search relevant data on the web.
* **image_generator_agent**: Agent that can generate images using Nova and save them to files.
* **Report Writing**: Report writer agent using Nova to create reports from the gather information.


By combining these components, our multi-agent system will be able to provide accurate and informative responses to a diverse set of queries.

## 1. Install Required Packages

First, let's install the necessary packages for our multi-agent system.

In [ ]:
!pip install -q -r requirements.txt --no-cache-dir

## 2. Setting Up API Keys and Environment

To access various services, such as Amazon Bedrock for Large Language Models (LLMs) and embedding models, we need to set up the necessary API keys and environment variables.

In [ ]:
import os
import sys
import boto3
import sagemaker
import json
import requests
import time
import datetime
import uuid
from typing import List, Dict, Any, Optional
import matplotlib.pyplot as plt
import networkx as nx

from strands import Agent, tool
from strands.models import BedrockModel
from strands_tools import file_read, file_write, editor, think, agent_graph

from IPython.display import display, Image
import logging


sts_client = boto3.client('sts')
account_id = sts_client.get_caller_identity()["Account"]
session = sagemaker.Session()

aws_region = boto3.session.Session().region_name

bedrock_model = BedrockModel(
    model_id="amazon.nova-premier-v1:0",
    region_name=aws_region
)

def display_image(image_path):
    import matplotlib.pyplot as plt
    from PIL import Image
    
    img = Image.open(image_path)
    plt.imshow(img)
    plt.axis('off')
    plt.show()
    
def extract_filename(response_string):
    import re
    
    # Convert to string if not already a string
    if not isinstance(response_string, str):
        response_string = str(response_string)
        
    pattern = r'<filename>\s*(.*?)\s*</filename>'
    matches = re.findall(pattern, response_string)
    return matches[-1] if matches else None



logging.getLogger('botocore.credentials').setLevel(logging.ERROR)
logging.getLogger('botocore').setLevel(logging.ERROR)
logging.getLogger('boto3').setLevel(logging.ERROR)


## 2. Creating individual Strands Agents

### get_stock_prices_agent

In [ ]:
from typing import Union, Dict, Set, List, TypedDict, Annotated
import pandas as pd
import yfinance as yf
from ta.momentum import RSIIndicator, StochasticOscillator
from ta.trend import SMAIndicator, EMAIndicator, MACD
from ta.volume import volume_weighted_average_price

#Define get_stock_prices_tool
@tool
def get_stock_prices_tool(ticker: str) -> Union[Dict, str]:
    """Fetches current and historical stock price data for a given ticker."""
    try:
        import datetime as dt
        import yfinance as yf
        
        # Get stock data
        stock = yf.Ticker(ticker)
        data = yf.download(
            ticker,
            start=dt.datetime.now() - dt.timedelta(days=90),
            end=dt.datetime.now(),
            interval='1d'
        )
        
        if data.empty:
            return f"No data found for ticker {ticker}"

        try:
            current_price = float(data['Close'].iloc[-1])
            previous_close = float(data['Close'].iloc[-2])
            current_volume = float(data['Volume'].iloc[-1])
            
            price_change = current_price - previous_close
            price_change_percent = (price_change / previous_close) * 100
            high_90d = float(data['High'].max())
            low_90d = float(data['Low'].min())
            avg_volume = float(data['Volume'].mean())

            return {
                "stock": ticker,
                "current_price": round(current_price, 2),
                "previous_close": round(previous_close, 2),
                "price_change": round(price_change, 2),
                "price_change_percent": round(price_change_percent, 2),
                "volume": int(current_volume),
                "high_90d": round(high_90d, 2),
                "low_90d": round(low_90d, 2),
                "average_volume": int(avg_volume),
                "date": dt.datetime.now().strftime("%Y-%m-%d")
            }

        except IndexError:
            return f"Insufficient data for ticker {ticker}"

    except Exception as e:
        return f"Error fetching price data: {str(e)}"

# # Define a specialized system prompt
GET_STOCK_PRICES_PROMPT = """
You are a specialized assistant to get the latest stock prices. Focus only on providing
factual, well-sourced information in response to stock prices related questions.
Always cite your sources when possible.
"""

# Define specialized agent to get stock prices
get_stock_prices_agent = Agent(
    system_prompt=GET_STOCK_PRICES_PROMPT,
    tools=[get_stock_prices_tool]  # search-specific tools
)


#Agent as tool
@tool
def get_stock_prices_agentt(query: str) -> str:
    try:
        # Strands Agents SDK makes it easy to create a specialized agent
        get_stock_prices_a =  Agent(
            system_prompt=GET_STOCK_PRICES_PROMPT,
            tools=[get_stock_prices_tool],
            callback_handler=None
        )

        # Call the agent and return its response
        response = get_stock_prices_a(query)
        return str(response)
    except Exception as e:
        return f"Error in get_stock_prices_a: {str(e)}"
   

### Testing get_stock_prices_agent

In [ ]:
#Test get_stock_prices_agent
while True:
    query = input("\nQuery> ")
    
    if query.lower() == "exit":
        print("\nGoodbye! 👋")
        break
            
    print("\nProcessing...\n")
    response = get_stock_prices_agent(query)
    print(response)
    print(f"Done!\n")


### fin_web_searcher_agent

In [ ]:
# Optional API keys for additional services
    # Tavily offers 1,000 free monthly credits for its AI-enhanced search API:
        # 1/ Go to app.tavily.com/sign-up
        # 2/ Authenticate with Google, GitHub, or email
        # 3/ Access API Keys in your dashboard
        # 4/ Copy your TAVILY_API_KEY

from tavily import TavilyClient


@tool
def search_web(query: str) -> Dict[str, Any]:
    # Using Tavily API for web search (you'll need an API key)
    TAVILY_API_KEY = "<YOUR API KEY>"
         
    client = TavilyClient(api_key=TAVILY_API_KEY)

    search_results=  client.search(query)   
    
    return search_results


bedrock_model = BedrockModel(
    model_config={
        "model_id": "us.anthropic.claude-sonnet-4-20250514-v1:0",
        "max_tokens": 256,
        "temperature": 0.1,
    }
)

# Define a specialized system prompt
FIN_WEB_SEARCHER_PROMPT = """ You are a financial researcher, focus on search and curated financial information. 
Focus only on providing factual, well-sourced information in response to financial questions.
Always cite your sources when possible.
"""

# Define specialized agent to get stock prices
fin_web_searcher_agent = Agent(
    system_prompt=FIN_WEB_SEARCHER_PROMPT,
    model= bedrock_model,
    tools=[search_web]  # search-specific tools
)


#Agent as tool
@tool
def fin_web_searcher_agentt(query: str) -> str:
    try:
        # Strands Agents SDK makes it easy to create a specialized agent
        fin_web_searcher_a = Agent(
            system_prompt=FIN_WEB_SEARCHER_PROMPT,
            model= bedrock_model,
            tools=[search_web],
            callback_handler=None
        )


        # Call the agent and return its response
        response = fin_web_searcher_a(query)
        return str(response)
    except Exception as e:
        return f"Error in fin_web_searcher_a: {str(e)}"
   

### Testing fin_web_searcher_agent

In [ ]:
#Test fin_web_searcher_agent
while True:
    query = input("\nQuery> ")
    
    if query.lower() == "exit":
        print("\nGoodbye! 👋")
        break
            
    print("\nProcessing...\n")
    response = fin_web_searcher_agent(query)
    print(response)
    print(f"Done!\n")

### image_generator_agent

In [ ]:
from img_generator import img_creator


SAVE_DIR = "./generated_images"
if not os.path.exists(SAVE_DIR):
    os.makedirs(SAVE_DIR)

IMAGE_GENERATOR_PROMPT="""You are an AI assistant that can generate images and save them to files.
    You can:
    1. Generate images using the img_creator tool
    2. Save files using the img_creator tool
    
    When users want to:
    - Generate an image: Use img_creator
    - Save the generated image: Use img_creator to save it
    - Both: First generate, then save the image

    If an image is generated, please provide the only filename as example:
    <filename>
        image.png
    </filename>
    Always confirm actions and provide clear feedback about what was done."""
    
# Create Image generation agent
image_generator_agent = Agent(
    system_prompt= IMAGE_GENERATOR_PROMPT,
    tools=[img_creator]
)            



#Agent as tool
@tool
def image_generator_agentt(query: str) -> str:
    try:
        # Strands Agents SDK makes it easy to create a specialized agent
        image_generator_a = Agent(
            system_prompt= IMAGE_GENERATOR_PROMPT,
            tools=[img_creator],
            callback_handler=None
        )  


        # Call the agent and return its response
        response = image_generator_a(query)
        response = image_generator_agent(query)
        filename = extract_filename(response)
        print(filename)
       
        if filename:
            display_image('./'+filename)
        return str(response)
    except Exception as e:
        return f"Error in image_generator_a: {str(e)}"

### Testing image_generator_agent

In [ ]:
#Test image_generator_agent
while True:
    query = input("\nQuery> ")
    
    if query.lower() == "exit":
        print("\nGoodbye! 👋")
        break
            
    print("\nProcessing...\n")
    response = image_generator_agent(query)
    filename = extract_filename(response)
    print(filename)
   
    if filename:
        display_image('./'+filename)
        
    
    print(response)
    print(f"Done!\n")

### report_writer_agent

In [ ]:
#report_writer_agent
REPORT_WRITER_PROMPT= """You are a professional report writing assistant.
        For financial reports:
        1. Create a well-structured report with the information provided
        2. Ensure professional tone and accuracy. 
        3. If required ask additional information to the coordinator agent
        4. Ask for user feedback and use it to modify/improve the report
        """

report_writer_agent = Agent(
    system_prompt=REPORT_WRITER_PROMPT,
    model=BedrockModel(
        model_id="us.amazon.nova-pro-v1:0",
        region=aws_region
    )
)


#Agent as tool
@tool
def report_writer_agentt(query: str) -> str:
    try:
        # Strands Agents SDK makes it easy to create a specialized agent
        report_writer_a = Agent(
            system_prompt=REPORT_WRITER_PROMPT,
            model=BedrockModel(
                model_id="us.amazon.nova-pro-v1:0",
                region=aws_region
            ),
            callback_handler=None
        )

        # Call the agent and return its response
        response = report_writer_a(query)
        return str(response)
    except Exception as e:
        return f"Error in report_writer_a: {str(e)}"

### Testing report_writer_agent

In [ ]:
#Test report_writer_agent

while True:
    query = input("\nQuery> ")
    
    if query.lower() == "exit":
        print("\nGoodbye! 👋")
        break
            
    print("\nProcessing...\n")
    response = report_writer_agent(query)
        
    
    print(response)
    print(f"Done!\n")

## 3. Creating Agent Graphs: Building Multi-Agent Systems

An agent graph is a structured network of interconnected AI agents designed to solve complex problems through coordinated collaboration. Each agent represents a specialized node with specific capabilities, and the connections between agents define explicit communication pathways.

Key components of an Agent Graph:

An agent graph consists of three primary components:

1. Nodes (Agents)¶
Nodes represent individual AI agents with:

    1. Identity: Unique identifier within the graph
    2. Role: Specialized function or purpose
    3. System Prompt: Instructions defining the agent's behavior
    4. Tools: Capabilities available to the agent
    5. Message Queue: Buffer for incoming communications
       
2. Edges (Connections)
Edges define the communication pathways between agents

3. Topology Patterns: Star, Mesh, Hierarchical

When to Use Agent Graphs
Agent graphs are ideal for:

1. Complex Communication Patterns: Custom topologies and interaction patterns
2. Persistent Agent State: Long-running agent networks that maintain context
3. Specialized Agent Roles: Different agents with distinct capabilities
4. Fine-Grained Control: Precise management of information flow

To learn more about Strands Agent Graphs: https://strandsagents.com/0.1.x/user-guide/concepts/multi-agent/graph/

## 3.1 Creating Agent Graphs: Building Star Topology Graph

We are going to create an agent graph using a star topology pattern. A central coordinator agent with radiating specialists, ideal for centralized workflows like content creation with editorial oversight or customer service with escalation paths. In our use case, we are going to create a coordinator agent to orchestrate the agents previously created.

In [ ]:
#Hierarchical Agent Graph Example
# Create the coordinator agent
COORD_AGENT_PROMPT="""You are a financial research team leader coordinating specialists.
                    Your job is to analyze the query and create a plan to answer it. Be concise and direct.
                    Use the node agents at your disposal to collect required data:
                    - To fetches current and historical stock price data for a given ticker, use get_stock_prices_agent
                    - To search for markets data and financial information, use fin_web_searcher_agent
                    - To generate imagens from text descriptions, use image_generator_agent
                    - To write a report based on all gather information, use report_writer_agent
                    Provide an objective and concise answer, based on all the data you gathered.
                    """
coord_agent = Agent(
    system_prompt=COORD_AGENT_PROMPT,
    tools=[get_stock_prices_agentt, fin_web_searcher_agentt, image_generator_agentt, report_writer_agentt],
    callback_handler=None
)

# Process a complex task through the hierarchical agent graph
def process_complex_task(task):
    """Process a complex task through the multi-level hierarchical agent graph"""
    return coord_agent(task)

### Testing Agent Graph

In [ ]:

while True:
    query = input("\nQuery> ")
    
    if query.lower() == "exit":
        print("\nGoodbye! 👋")
        break
            
    print("\nProcessing...\n")
    response = process_complex_task(query)
    print(response)
    print(f"Done!\n")

#Analyze Amazon's stock performance over the last quarter and create a report with visualizations
#give me a summary report of amazon earnings of Q1 2025

## 4. Conclusion

In this notebook, we've built a multi-agent system using Strands that can handle a variety of query types by routing them to specialized agents. The system demonstrates how to effectively orchestrate multiple AI agents to provide accurate and informative responses to diverse user queries.